# Python Beyond Basics (Lesson 8)

> 继承 和 多态

## Simple Inheritence

class SubClass(BaseClass)

* SubClass will want to initialize base classes
* BaseClass initializer will only be called automatically if **subclass intializer is undefined**

In [2]:
class Base:
    
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')

In [3]:
b = Base()

Base initializer


In [4]:
b.f()

Base.f()


In [6]:
class Sub(Base):
    pass

In [7]:
s = Sub()

Base initializer


In [8]:
s.f()

Base.f()


重写 Sub 的 f 方法:

In [12]:
class Sub(Base):
    
    def f(self):
        print('Sub.f()')

In [13]:
s = Sub()

Base initializer


In [14]:
s.f()

Sub.f()


重写 Sub 的 `__init__` 方法:

In [15]:
class Sub(Base):
    
    def __init__(self):
        print('Sub initializer')
    
    def f(self):
        print('Sub.f()')

In [16]:
s = Sub()

Sub initializer


In [17]:
s.f()

Sub.f()


Python的继承：

如果被重写了`__init__()` 不会自动调用 Base 的 `__init()__` ,需要手动 `super().__init__()`

![Python Inheritence](https://i.loli.net/2018/01/10/5a55d32d08fbf.png)


In [18]:
class Sub(Base):
    
    def __init__(self):
        super().__init__()
        print('Sub initializer')
    
    def f(self):
        print('Sub.f()')

In [19]:
s = Sub()

Base initializer
Sub initializer


### SimpleList

In [68]:
class SimpleList:
    
    def __init__(self, items):
        self._items = list(items)
        
    def add(self, item):
        self._items.append(item)
        
    def __getitem__(self, idx):
        return self._items[idx]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return 'SimpleList({!r})'.format(list(self))

In [69]:
class SortedList(SimpleList):
    
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
        
    def add(self, item):
        super().add(item)
        self.sort()
        
    def __repr__(self):
        return 'SortedList({!r})'.format(list(self))

Tips: 多次 用 super() 去掉用父类的Method(base-class implemention)

In [70]:
sl = SortedList([4, 3, 7, 9])

In [71]:
sl

SortedList([3, 4, 7, 9])

In [72]:
sl.add(0)

In [73]:
sl

SortedList([0, 3, 4, 7, 9])

### Multiple Inheritence

Multiple Inheritence in Python is **not much more complex** than single inheritence


isinstance() 
* determins if an object is a specified type
* Use isinstance() for runtime type checking

In [74]:
 isinstance(3, int)

True

In [75]:
 isinstance('hello', str)

True

In [76]:
isinstance(sl, SortedList)

True

In [77]:
isinstance(sl, SimpleList)

True

In [78]:
x = []
isinstance(x, (list, dict, tuple))

True

In [79]:
# 加入 item 的类型判断

class IntList(SimpleList):
    
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
    
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only support int values')
    
    def add(self, item):
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return 'IntList({!r})'.format(list(self))

In [80]:
il = IntList([1, 2, 3])
il.add(5)

In [81]:
il

IntList([1, 2, 3, 5])

In [82]:
il.add('6')

TypeError: IntList only support int values

issubclass() - determines if one type is a subclass of another

In [ ]:
issubclass(IntList, SimpleList)

In [ ]:
issubclass(IntList, SortedList)

In [ ]:
issubclass(SimpleList, object)

In [ ]:
class MyInt(int): pass
class MyVerySpecialInt(MyInt): pass

issubclass(MyVerySpecialInt, int)

### Multiple Inheritence

> (C++ 支持， Java不支持)

defining a class with more than one base class

class SubClass(Base1, Base2,...)

* SubClass inherits methods of all bases
* Without conflict, name resolve in the obvious way
* Method Resolution Order(MRO) determines name lookup in all cases

In [ ]:
class SortedIntList(IntList, SortedList):
    
    # 注意没有定义 add(), __init__(), 调用的时候 遵守 MRO
    
    def __repr__(self):
        return 'SortedIntList({!r})'.format(list(self))

In [83]:
# 不支持 str

SortedIntList([1, 2, '3'])

TypeError: IntList only support int values

In [89]:
sil = SortedIntList([3, 5, 1])
sil

IntList([1, 3, 5])

In [90]:
sil.add('6')

TypeError: IntList only support int values

In [91]:
sil.add(6)
sil

IntList([1, 3, 5, 6])